In [1]:
from pyspark.sql import SparkSession

# Crear una sesión de Spark
spark = SparkSession.builder.appName("CSVtoParquet_vacunascovidver2").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/27 19:38:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
# Leer los archivos CSV

# Leer el primer archivo
df_2021_parte1 = spark.read \
    .format("csv") \
    .option("delimiter", ",") \
    .option("header", "true") \
    .option("encoding", "latin1") \
    .load("/rawdata/vacunascovid/vacunaci_nCOVID-19_2021_parte1.txt")

# Leer el segundo archivo
df_2021_parte2 = spark.read \
    .format("csv") \
    .option("delimiter", ",") \
    .option("header", "true") \
    .option("encoding", "latin1") \
    .load("/rawdata/vacunascovid/vacunaci_nCOVID-19_2021_parte2.txt")

# Leer el tercer archivo
df_2021_parte3 = spark.read \
    .format("csv") \
    .option("delimiter", ",") \
    .option("header", "true") \
    .option("encoding", "latin1") \
    .load("/rawdata/vacunascovid/vacunaci_nCOVID-19_2021_parte3.txt")

# Leer el cuarto archivo
df_2022 = spark.read \
    .format("csv") \
    .option("delimiter", ",") \
    .option("header", "true") \
    .option("encoding", "latin1") \
    .load("/rawdata/vacunascovid/vacunaci_nCOVID-19_2022.txt")

In [2]:
# Configurar tamaño objetivo de los archivos Parquet (aproximadamente 7MB)
target_file_size_mb = 7
target_file_size_bytes = target_file_size_mb * 1024 * 1024

# Definir una función para guardar DataFrame como Parquet con tamaño objetivo
def save_as_parquet(df, output_dir, target_size):
    # Repartir el DataFrame para aproximar el tamaño objetivo
    df = df.repartition((df.count() * df.columns.__len__() * 8) // target_size)
    # Escribir el DataFrame en el formato Parquet
    df.write.mode("overwrite").parquet(output_dir)

In [5]:
df_unido_2021 = df_2021_parte1.union(df_2021_parte2).union(df_2021_parte3)

In [6]:
df_unido_2021.count()

59904692

In [7]:
# Guardar el primer DataFrame 2021 como archivos Parquet en la carpeta especificada
save_as_parquet(df_unido_2021, "/rawdata/vacunascovid/vacunascovid_ver2_parquet_2021", target_file_size_bytes)

24/06/27 19:56:48 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [8]:
# Guardar el segundo DataFrame como archivos Parquet en la carpeta especificada
save_as_parquet(df_2022, "/rawdata/vacunascovid/vacunascovid_ver2_parquet_2022", target_file_size_bytes)

In [ ]:
import subprocess

# Función para renombrar archivos Parquet
def rename_parquet_files(directory):
    # Listar los archivos en el directorio
    files = subprocess.check_output(["hdfs", "dfs", "-ls", directory]).decode("utf-8").split("\n")
    parquet_files = [line.split()[-1] for line in files if line.endswith(".parquet")]
    
    # Renombrar los archivos
    for idx, file in enumerate(parquet_files):
        new_name = f"{directory}/part.{idx}.parquet"
        subprocess.call(["hdfs", "dfs", "-mv", file, new_name])

# Renombrar los archivos en las carpetas especificadas
rename_parquet_files("/rawdata/vacunascovid/vacunascovid_ver2_parquet_2021")
rename_parquet_files("/rawdata/vacunascovid/vacunascovid_ver2_parquet_2022")

2024-06-27 20:11:17,538 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2024-06-27 20:11:19,202 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2024-06-27 20:11:20,614 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2024-06-27 20:11:22,033 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2024-06-27 20:11:23,910 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2024-06-27 20:11:25,238 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2024-06-27 20:11:26,627 WARN util.NativeCodeLoader: Unable to load native-ha

2024-06-27 20:12:35,890 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2024-06-27 20:12:37,243 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2024-06-27 20:12:38,645 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2024-06-27 20:12:40,014 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2024-06-27 20:12:41,430 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2024-06-27 20:12:42,764 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2024-06-27 20:12:44,132 WARN util.NativeCodeLoader: Unable to load native-ha

2024-06-27 20:13:52,718 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2024-06-27 20:13:54,098 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2024-06-27 20:13:55,664 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2024-06-27 20:13:57,234 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2024-06-27 20:13:58,677 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2024-06-27 20:14:00,070 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2024-06-27 20:14:01,639 WARN util.NativeCodeLoader: Unable to load native-ha

2024-06-27 20:15:09,433 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2024-06-27 20:15:10,764 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2024-06-27 20:15:12,132 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2024-06-27 20:15:13,492 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2024-06-27 20:15:14,883 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2024-06-27 20:15:16,272 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2024-06-27 20:15:17,676 WARN util.NativeCodeLoader: Unable to load native-ha

2024-06-27 20:16:29,395 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2024-06-27 20:16:30,764 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2024-06-27 20:16:32,104 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2024-06-27 20:16:33,494 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2024-06-27 20:16:34,863 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2024-06-27 20:16:36,232 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2024-06-27 20:16:37,624 WARN util.NativeCodeLoader: Unable to load native-ha

...

2024-06-27 21:38:31,412 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2024-06-27 21:38:32,818 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2024-06-27 21:38:34,213 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2024-06-27 21:38:35,568 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2024-06-27 21:38:36,904 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2024-06-27 21:38:38,534 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2024-06-27 21:38:39,936 WARN util.NativeCodeLoader: Unable to load native-ha

2024-06-27 21:39:56,930 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2024-06-27 21:39:58,294 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2024-06-27 21:39:59,682 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2024-06-27 21:40:01,054 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2024-06-27 21:40:02,412 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2024-06-27 21:40:03,805 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2024-06-27 21:40:05,204 WARN util.NativeCodeLoader: Unable to load native-ha

2024-06-27 21:41:20,775 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2024-06-27 21:41:22,583 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2024-06-27 21:41:23,979 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2024-06-27 21:41:25,349 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2024-06-27 21:41:26,707 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2024-06-27 21:41:28,119 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2024-06-27 21:41:29,493 WARN util.NativeCodeLoader: Unable to load native-ha

2024-06-27 21:42:40,328 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2024-06-27 21:42:41,933 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2024-06-27 21:42:43,375 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2024-06-27 21:42:44,722 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2024-06-27 21:42:46,063 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2024-06-27 21:42:49,304 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2024-06-27 21:42:50,662 WARN util.NativeCodeLoader: Unable to load native-ha

2024-06-27 21:44:01,893 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2024-06-27 21:44:03,255 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2024-06-27 21:44:04,613 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2024-06-27 21:44:05,985 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2024-06-27 21:44:07,834 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2024-06-27 21:44:09,442 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2024-06-27 21:44:10,823 WARN util.NativeCodeLoader: Unable to load native-ha

In [ ]:
# Detener la sesión de Spark
spark.stop()